In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style("whitegrid")

os.makedirs("model", exist_ok=True)

df = pd.read_csv("data/n_movies.csv")

print(f"Загружено фильмов: {len(df)}")
df.head()

Загружено фильмов: 9957


,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"


In [ ]:
print("Информация о датасете:")
print(df.info())

print("\nОсновные статистики:")
print(df.describe())

print("\nПропущенные значения:")
print(df.isnull().sum())

print("\nТоп-10 фильмов по рейтингу:")
print(df.nlargest(10, 'rating')[['title', 'year', 'rating', 'genre']])

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['rating'], bins=30, kde=True, color='#4C72B0')
plt.title('Распределение рейтингов фильмов', fontsize=16)
plt.xlabel('Рейтинг')
plt.ylabel('Количество фильмов')
plt.axvline(df['rating'].mean(), color='red', linestyle='--', label=f"Средний рейтинг: {df['rating'].mean():.2f}")
plt.legend()
plt.show()

In [ ]:
df['year_num'] = df['year'].str.extract(r'(\d{4})').astype(float)

plt.figure(figsize=(14, 6))
sns.histplot(df['year_num'].dropna(), bins=40, color='#55A868')
plt.title('Распределение фильмов по годам выпуска', fontsize=16)
plt.xlabel('Год')
plt.ylabel('Количество фильмов')
plt.show()

print(f"Самый старый фильм: {df['year_num'].min()}")
print(f"Самый новый фильм: {df['year_num'].max()}")

In [ ]:
all_genres = df['genre'].str.split(', ').explode()
top_genres = all_genres.value_counts().head(15)

plt.figure(figsize=(12, 8))
top_genres.plot(kind='barh', color='orange')
plt.title('Самые популярные жанры (топ-15)')
plt.xlabel('Количество фильмов')
plt.ylabel('Жанр')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
all_genres = df['genre'].str.split(', ').explode() 
genre_freq = all_genres.value_counts() 
text = " ".join(all_genres.dropna())

wordcloud = WordCloud(
    width=1000, height=600,
    background_color='white',
    colormap='viridis', 
    max_words=100,
    relative_scaling=0.5
).generate_from_frequencies(genre_freq) 

plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Облако жанров — каждое слово только один раз, размер = популярность')
plt.show()

In [ ]:
df = df.drop_duplicates(subset="title").reset_index(drop=True)

df["genre"] = df["genre"].fillna("")
df["description"] = df["description"].fillna("")
df["stars"] = df["stars"].fillna("")

df["features"] = (
    df["genre"] + " " + 
    df["stars"].str.replace(r'[\[\]\'\",]', ' ', regex=True) + " " + 
    df["description"]
)
df["features"] = df["features"].str.replace(r'\s+', ' ', regex=True).str.strip()

print(f"После очистки осталось: {len(df)} фильмов")
df[["title", "features"]].head()

После очистки осталось: 7912 фильмов


,title,features
0,Cobra Kai,"Action, Comedy, Drama Ralph Macchio William Za..."
1,The Crown,"Biography, Drama, History Claire Foy Olivia Co..."
2,Better Call Saul,"Crime, Drama Bob Odenkirk Rhea Seehorn Jonatha..."
3,Devil in Ohio,"Drama, Horror, Mystery Emily Deschanel Sam Jae..."
4,Cyberpunk: Edgerunners,"Animation, Action, Adventure Zach Aguilar Keni..."


In [6]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df["features"])

similarity_matrix = cosine_similarity(tfidf_matrix)

print("Модель готова!")
print(f"Матрица сходства: {similarity_matrix.shape}")

Модель готова!
Матрица сходства: (7912, 7912)


In [7]:
np.save("model/similarity_matrix.npy", similarity_matrix)
pickle.dump(df["title"].tolist(), open("model/titles.pkl", "wb"))
pickle.dump(vectorizer, open("model/vectorizer.pkl", "wb"))
df.to_pickle("model/movies_data.pkl")

print("Всё сохранено в папку model!")

Всё сохранено в папку model!


In [ ]:
def test_recommend(title):
    if title not in df["title"].values:
        print(f"Фильм '{title}' не найден")
        return
    idx = df[df["title"] == title].index[0]
    scores = similarity_matrix[idx]
    top_idx = scores.argsort()[-7:-1][::-1]
    print(f"\nРекомендации для: {title}")
    print(df.iloc[top_idx][["title", "year", "rating"]].to_string(index=False))

test_recommend("Cobra Kai")
test_recommend("Breaking Bad")